<a href="https://colab.research.google.com/github/Gustavens/Google-colab-AI/blob/main/Fase_3_T3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
import matplotlib.pyplot as plt

In [21]:
import os
import zipfile

file_path = '/content/vehicle_dataset.zip'

if os.path.exists(file_path):
    print(f"File '{file_path}' exists.")
    # Proceed with unzipping
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall('/content/')
else:
    print(f"File '{file_path}' does not exist. Please check the path.")

File '/content/vehicle_dataset.zip' exists.


In [23]:
# Definir las rutas a tus carpetas de datos (¡ACTUALIZA ESTAS RUTAS!)
train_dir = '/content/vehicle_dataset/train'  # Reemplaza con la ruta correcta
validation_dir = '/content/vehicle_dataset/validation' # Reemplaza con la ruta correcta
test_dir = '/content/vehicle_dataset/test'    # Reemplaza con la ruta correcta

# Definir el tamaño de las imágenes que espera el modelo MobileNetV2
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [24]:
# Crear los conjuntos de datos usando image_dataset_from_directory
train_dataset = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical', # Codificación one-hot de las etiquetas
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

validation_dataset = image_dataset_from_directory(
    validation_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_dataset = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Visualizar las clases y el número de batches
class_names = train_dataset.class_names
print("Clases:", class_names)
print("Número de batches en el conjunto de entrenamiento:", train_dataset.cardinality().numpy())
print("Número de batches en el conjunto de validación:", validation_dataset.cardinality().numpy())
print("Número de batches en el conjunto de prueba:", test_dataset.cardinality().numpy())

# Preprocesar los datos aplicando la función preprocess_input específica de MobileNetV2
def preprocess(image, label):
    return preprocess_input(image), label

train_dataset = train_dataset.map(preprocess)
validation_dataset = validation_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

# Opcional: Optimizar el rendimiento del cargador de datos
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

Found 368 files belonging to 4 classes.
Found 132 files belonging to 4 classes.
Found 299 files belonging to 4 classes.
Clases: ['golpes', 'raspones', 'sin_daños', 'vidrios_rotos']
Número de batches en el conjunto de entrenamiento: 12
Número de batches en el conjunto de validación: 5
Número de batches en el conjunto de prueba: 10


El siguiente paso es cargar el modelo preentrenado MobileNetV2 y luego agregar capas para adaptarlo a las de 4 clases mencionadas.

In [25]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [26]:
# Definir el modelo base preentrenado (MobileNetV2 sin la capa superior)
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Congelar las capas del modelo base para que no se entrenen
for layer in base_model.layers:
    layer.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [30]:
# Agregar nuevas capas de clasificación
x = base_model.output
x = Flatten()(x)
#Aplanar: Convierte las características multidimensionales en un vector.
#Capa Densa: Agrega una capa completamente conectada para aprender combinaciones complejas de las características.
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x) # Regularización para evitar el sobreajuste
predictions = Dense(len(class_names), activation='softmax')(x) # Capa de salida con 4 neuronas (una por clase) y softmax
# Genera las probabilidades de pertenencia a cada una de las clases definidas en class_names

In [28]:
# Crear el modelo final combinando el modelo base y las nuevas capas
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0001), # Una tasa de aprendizaje baja para el fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
# Mostrar la arquitectura del modelo
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 10,286,788 (39.24 MB)

 Trainable params: 8,028,804 (30.63 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [31]:
# Entrenar el modelo con una instancia de prueba de 20 epocas
epochs = 20


In [ ]:
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=validation_dataset
)

Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 50s 4s/step - accuracy: 0.5203 - loss: 2.5137 - val_accuracy: 0.7273 - val_loss: 0.6418
Epoch 2/20
 6/12 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.9067 - loss: 0.3134 

Visualización del modelo

In [ ]:
# Visualizar el rendimiento del entrenamiento
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()